# Numpy Project - Part 4: 3D Sudokus! Increasing dimensions.

Now it's time to increase the number of dimensions of our arrays. We'll use a public [Kaggle Dataset](https://www.kaggle.com/bryanpark/sudoku) that contains 1 million Sudoku games!

We've reduced the total dataset to 5000 games for simplicity, but it'll still be fun. Let's get started

In [1]:
import numpy as np

First let's take a look at the structure of the CSV file:

In [6]:
!head sudoku-small.csv

quizzes,solutions
004300209005009001070060043006002087190007400050083000600000105003508690042910300,864371259325849761971265843436192587198657432257483916689734125713528694542916378
040100050107003960520008000000000017000906800803050620090060543600080700250097100,346179258187523964529648371965832417472916835813754629798261543631485792254397186
600120384008459072000006005000264030070080006940003000310000050089700000502000190,695127384138459672724836915851264739273981546946573821317692458489715263562348197
497200000100400005000016098620300040300900000001072600002005870000600004530097061,497258316186439725253716498629381547375964182841572639962145873718623954534897261
005910308009403060027500100030000201000820007006007004000080000640150700890000420,465912378189473562327568149738645291954821637216397854573284916642159783891736425
100005007380900000600000480820001075040760020069002001005039004000020100000046352,19468523738297451665721348982349167554176892376935284121583976443652719897814635

In [7]:
!wc -l sudoku-small.csv

5000 sudoku-small.csv


As you can see, it's a very simple CSV containing only 2 columns, the empty board, and the solution. The way the board is expressed is different though; in this case it's just a long string containing all the numbers.

### 1) Parsing long string lines into valid boards

We need to adapt to this new style of expressing Sudoku boards. This is a valuable lesson in data handling: you can't anticipate all the different ways that there will be to express data. It'd be a mistake to extend the `Board` class also including this way of expressing puzzles; we try not to modify our core data structures adding edge cases; instead, we'll write an _"adapter"_ (see [Wikipedia's article about the Software Pattern](https://en.wikipedia.org/wiki/Adapter_pattern)), which is just a tiny function that will turn the long puzzle line into a numpy array:

In [8]:
# Solution
def adapt_long_sudoku_line_to_array(line):
    return np.array([int(c) for c in line]).reshape(9, 9)

In [9]:
adapt_long_sudoku_line_to_array('004300209005009001070060043006002087190007400050083000600000105003508690042910300')

array([[0, 0, 4, 3, 0, 0, 2, 0, 9],
       [0, 0, 5, 0, 0, 9, 0, 0, 1],
       [0, 7, 0, 0, 6, 0, 0, 4, 3],
       [0, 0, 6, 0, 0, 2, 0, 8, 7],
       [1, 9, 0, 0, 0, 7, 4, 0, 0],
       [0, 5, 0, 0, 8, 3, 0, 0, 0],
       [6, 0, 0, 0, 0, 0, 1, 0, 5],
       [0, 0, 3, 5, 0, 8, 6, 9, 0],
       [0, 4, 2, 9, 1, 0, 3, 0, 0]])

In [10]:
line = '004300209005009001070060043006002087190007400050083000600000105003508690042910300'

In [11]:
assert np.array_equal(adapt_long_sudoku_line_to_array(line), np.array([
    [0, 0, 4, 3, 0, 0, 2, 0, 9],
    [0, 0, 5, 0, 0, 9, 0, 0, 1],
    [0, 7, 0, 0, 6, 0, 0, 4, 3],
    [0, 0, 6, 0, 0, 2, 0, 8, 7],
    [1, 9, 0, 0, 0, 7, 4, 0, 0],
    [0, 5, 0, 0, 8, 3, 0, 0, 0],
    [6, 0, 0, 0, 0, 0, 1, 0, 5],
    [0, 0, 3, 5, 0, 8, 6, 9, 0],
    [0, 4, 2, 9, 1, 0, 3, 0, 0]
]))


### 2) Reading a CSV file into a 3-dimensional array

Now it's time to read multiple sudoku puzzles into a single Numpy array. We'll end up with a 3-dimensional array, the first 2 dimensions (x, y) are the ones of a puzzle, and the 3rd dimension (z) is for multiple puzzles. Here's a graphical representation of it:

<img width="600px" src="https://user-images.githubusercontent.com/872296/68670705-499dce00-052c-11ea-8e82-18a1f435e274.png">


For example, we want to create something like this:

In [12]:
np.array([
    [
        [0, 0, 4, 3, 0, 0, 2, 0, 9],
        [0, 0, 5, 0, 0, 9, 0, 0, 1],
        [0, 7, 0, 0, 6, 0, 0, 4, 3],
        [0, 0, 6, 0, 0, 2, 0, 8, 7],
        [1, 9, 0, 0, 0, 7, 4, 0, 0],
        [0, 5, 0, 0, 8, 3, 0, 0, 0],
        [6, 0, 0, 0, 0, 0, 1, 0, 5],
        [0, 0, 3, 5, 0, 8, 6, 9, 0],
        [0, 4, 2, 9, 1, 0, 3, 0, 0]
    ],
    [
        [0, 0, 4, 3, 0, 0, 2, 0, 9],
        [0, 0, 5, 0, 0, 9, 0, 0, 1],
        [0, 7, 0, 0, 6, 0, 0, 4, 3],
        [0, 0, 6, 0, 0, 2, 0, 8, 7],
        [1, 9, 0, 0, 0, 7, 4, 0, 0],
        [0, 5, 0, 0, 8, 3, 0, 0, 0],
        [6, 0, 0, 0, 0, 0, 1, 0, 5],
        [0, 0, 3, 5, 0, 8, 6, 9, 0],
        [0, 4, 2, 9, 1, 0, 3, 0, 0]
    ],
    [
        [0, 0, 4, 3, 0, 0, 2, 0, 9],
        [0, 0, 5, 0, 0, 9, 0, 0, 1],
        [0, 7, 0, 0, 6, 0, 0, 4, 3],
        [0, 0, 6, 0, 0, 2, 0, 8, 7],
        [1, 9, 0, 0, 0, 7, 4, 0, 0],
        [0, 5, 0, 0, 8, 3, 0, 0, 0],
        [6, 0, 0, 0, 0, 0, 1, 0, 5],
        [0, 0, 3, 5, 0, 8, 6, 9, 0],
        [0, 4, 2, 9, 1, 0, 3, 0, 0]
    ],
])

array([[[0, 0, 4, 3, 0, 0, 2, 0, 9],
        [0, 0, 5, 0, 0, 9, 0, 0, 1],
        [0, 7, 0, 0, 6, 0, 0, 4, 3],
        [0, 0, 6, 0, 0, 2, 0, 8, 7],
        [1, 9, 0, 0, 0, 7, 4, 0, 0],
        [0, 5, 0, 0, 8, 3, 0, 0, 0],
        [6, 0, 0, 0, 0, 0, 1, 0, 5],
        [0, 0, 3, 5, 0, 8, 6, 9, 0],
        [0, 4, 2, 9, 1, 0, 3, 0, 0]],

       [[0, 0, 4, 3, 0, 0, 2, 0, 9],
        [0, 0, 5, 0, 0, 9, 0, 0, 1],
        [0, 7, 0, 0, 6, 0, 0, 4, 3],
        [0, 0, 6, 0, 0, 2, 0, 8, 7],
        [1, 9, 0, 0, 0, 7, 4, 0, 0],
        [0, 5, 0, 0, 8, 3, 0, 0, 0],
        [6, 0, 0, 0, 0, 0, 1, 0, 5],
        [0, 0, 3, 5, 0, 8, 6, 9, 0],
        [0, 4, 2, 9, 1, 0, 3, 0, 0]],

       [[0, 0, 4, 3, 0, 0, 2, 0, 9],
        [0, 0, 5, 0, 0, 9, 0, 0, 1],
        [0, 7, 0, 0, 6, 0, 0, 4, 3],
        [0, 0, 6, 0, 0, 2, 0, 8, 7],
        [1, 9, 0, 0, 0, 7, 4, 0, 0],
        [0, 5, 0, 0, 8, 3, 0, 0, 0],
        [6, 0, 0, 0, 0, 0, 1, 0, 5],
        [0, 0, 3, 5, 0, 8, 6, 9, 0],
        [0, 4, 2, 9, 1, 0, 3, 0, 0

Now it's time to code! Complete the function `read_sudokus_from_csv`; it receives two parameters, the name of the `csv` file to read and an optional one `read_solutions`. If `read_solutions` is True, you're supposed to read from the second column (solutions) instead of empty puzzles. You can assume the following CSV structure:

```
quizzes,solutions
10084..,183048..
30018..,34196..
...
empty,solved
empty,solved
```

In [13]:
# Solution
import csv
def read_sudokus_from_csv(filename, read_solutions=False):
    index = 0 if not read_solutions else 1
    with open(filename) as fp:
        reader = csv.reader(fp)
        next(reader)  # Drop the header
        puzzles = [
            adapt_long_sudoku_line_to_array(line[index])
            for line in reader
        ]
        return np.array(puzzles)

For this test we'll use the file `sudoku-micro.csv` that contains only 3 puzzles:

In [14]:
read_sudokus_from_csv('sudoku-micro.csv')

array([[[0, 0, 4, 3, 0, 0, 2, 0, 9],
        [0, 0, 5, 0, 0, 9, 0, 0, 1],
        [0, 7, 0, 0, 6, 0, 0, 4, 3],
        [0, 0, 6, 0, 0, 2, 0, 8, 7],
        [1, 9, 0, 0, 0, 7, 4, 0, 0],
        [0, 5, 0, 0, 8, 3, 0, 0, 0],
        [6, 0, 0, 0, 0, 0, 1, 0, 5],
        [0, 0, 3, 5, 0, 8, 6, 9, 0],
        [0, 4, 2, 9, 1, 0, 3, 0, 0]],

       [[0, 4, 0, 1, 0, 0, 0, 5, 0],
        [1, 0, 7, 0, 0, 3, 9, 6, 0],
        [5, 2, 0, 0, 0, 8, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 7],
        [0, 0, 0, 9, 0, 6, 8, 0, 0],
        [8, 0, 3, 0, 5, 0, 6, 2, 0],
        [0, 9, 0, 0, 6, 0, 5, 4, 3],
        [6, 0, 0, 0, 8, 0, 7, 0, 0],
        [2, 5, 0, 0, 9, 7, 1, 0, 0]],

       [[6, 0, 0, 1, 2, 0, 3, 8, 4],
        [0, 0, 8, 4, 5, 9, 0, 7, 2],
        [0, 0, 0, 0, 0, 6, 0, 0, 5],
        [0, 0, 0, 2, 6, 4, 0, 3, 0],
        [0, 7, 0, 0, 8, 0, 0, 0, 6],
        [9, 4, 0, 0, 0, 3, 0, 0, 0],
        [3, 1, 0, 0, 0, 0, 0, 5, 0],
        [0, 8, 9, 7, 0, 0, 0, 0, 0],
        [5, 0, 2, 0, 0, 0, 1, 9, 0

In [15]:
expected = np.array([[[0, 0, 4, 3, 0, 0, 2, 0, 9],
        [0, 0, 5, 0, 0, 9, 0, 0, 1],
        [0, 7, 0, 0, 6, 0, 0, 4, 3],
        [0, 0, 6, 0, 0, 2, 0, 8, 7],
        [1, 9, 0, 0, 0, 7, 4, 0, 0],
        [0, 5, 0, 0, 8, 3, 0, 0, 0],
        [6, 0, 0, 0, 0, 0, 1, 0, 5],
        [0, 0, 3, 5, 0, 8, 6, 9, 0],
        [0, 4, 2, 9, 1, 0, 3, 0, 0]],

       [[0, 4, 0, 1, 0, 0, 0, 5, 0],
        [1, 0, 7, 0, 0, 3, 9, 6, 0],
        [5, 2, 0, 0, 0, 8, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 7],
        [0, 0, 0, 9, 0, 6, 8, 0, 0],
        [8, 0, 3, 0, 5, 0, 6, 2, 0],
        [0, 9, 0, 0, 6, 0, 5, 4, 3],
        [6, 0, 0, 0, 8, 0, 7, 0, 0],
        [2, 5, 0, 0, 9, 7, 1, 0, 0]],

       [[6, 0, 0, 1, 2, 0, 3, 8, 4],
        [0, 0, 8, 4, 5, 9, 0, 7, 2],
        [0, 0, 0, 0, 0, 6, 0, 0, 5],
        [0, 0, 0, 2, 6, 4, 0, 3, 0],
        [0, 7, 0, 0, 8, 0, 0, 0, 6],
        [9, 4, 0, 0, 0, 3, 0, 0, 0],
        [3, 1, 0, 0, 0, 0, 0, 5, 0],
        [0, 8, 9, 7, 0, 0, 0, 0, 0],
        [5, 0, 2, 0, 0, 0, 1, 9, 0]]])

In [16]:
assert np.array_equal(read_sudokus_from_csv('sudoku-micro.csv'), expected)

Reading solutions:

In [17]:
read_sudokus_from_csv('sudoku-micro.csv', read_solutions=True)

array([[[8, 6, 4, 3, 7, 1, 2, 5, 9],
        [3, 2, 5, 8, 4, 9, 7, 6, 1],
        [9, 7, 1, 2, 6, 5, 8, 4, 3],
        [4, 3, 6, 1, 9, 2, 5, 8, 7],
        [1, 9, 8, 6, 5, 7, 4, 3, 2],
        [2, 5, 7, 4, 8, 3, 9, 1, 6],
        [6, 8, 9, 7, 3, 4, 1, 2, 5],
        [7, 1, 3, 5, 2, 8, 6, 9, 4],
        [5, 4, 2, 9, 1, 6, 3, 7, 8]],

       [[3, 4, 6, 1, 7, 9, 2, 5, 8],
        [1, 8, 7, 5, 2, 3, 9, 6, 4],
        [5, 2, 9, 6, 4, 8, 3, 7, 1],
        [9, 6, 5, 8, 3, 2, 4, 1, 7],
        [4, 7, 2, 9, 1, 6, 8, 3, 5],
        [8, 1, 3, 7, 5, 4, 6, 2, 9],
        [7, 9, 8, 2, 6, 1, 5, 4, 3],
        [6, 3, 1, 4, 8, 5, 7, 9, 2],
        [2, 5, 4, 3, 9, 7, 1, 8, 6]],

       [[6, 9, 5, 1, 2, 7, 3, 8, 4],
        [1, 3, 8, 4, 5, 9, 6, 7, 2],
        [7, 2, 4, 8, 3, 6, 9, 1, 5],
        [8, 5, 1, 2, 6, 4, 7, 3, 9],
        [2, 7, 3, 9, 8, 1, 5, 4, 6],
        [9, 4, 6, 5, 7, 3, 8, 2, 1],
        [3, 1, 7, 6, 9, 2, 4, 5, 8],
        [4, 8, 9, 7, 1, 5, 2, 6, 3],
        [5, 6, 2, 3, 4, 8, 1, 9, 7

In [18]:
expected = np.array([[[8, 6, 4, 3, 7, 1, 2, 5, 9],
        [3, 2, 5, 8, 4, 9, 7, 6, 1],
        [9, 7, 1, 2, 6, 5, 8, 4, 3],
        [4, 3, 6, 1, 9, 2, 5, 8, 7],
        [1, 9, 8, 6, 5, 7, 4, 3, 2],
        [2, 5, 7, 4, 8, 3, 9, 1, 6],
        [6, 8, 9, 7, 3, 4, 1, 2, 5],
        [7, 1, 3, 5, 2, 8, 6, 9, 4],
        [5, 4, 2, 9, 1, 6, 3, 7, 8]],

       [[3, 4, 6, 1, 7, 9, 2, 5, 8],
        [1, 8, 7, 5, 2, 3, 9, 6, 4],
        [5, 2, 9, 6, 4, 8, 3, 7, 1],
        [9, 6, 5, 8, 3, 2, 4, 1, 7],
        [4, 7, 2, 9, 1, 6, 8, 3, 5],
        [8, 1, 3, 7, 5, 4, 6, 2, 9],
        [7, 9, 8, 2, 6, 1, 5, 4, 3],
        [6, 3, 1, 4, 8, 5, 7, 9, 2],
        [2, 5, 4, 3, 9, 7, 1, 8, 6]],

       [[6, 9, 5, 1, 2, 7, 3, 8, 4],
        [1, 3, 8, 4, 5, 9, 6, 7, 2],
        [7, 2, 4, 8, 3, 6, 9, 1, 5],
        [8, 5, 1, 2, 6, 4, 7, 3, 9],
        [2, 7, 3, 9, 8, 1, 5, 4, 6],
        [9, 4, 6, 5, 7, 3, 8, 2, 1],
        [3, 1, 7, 6, 9, 2, 4, 5, 8],
        [4, 8, 9, 7, 1, 5, 2, 6, 3],
        [5, 6, 2, 3, 4, 8, 1, 9, 7]]])

In [19]:
assert np.array_equal(read_sudokus_from_csv('sudoku-micro.csv', read_solutions=True), expected)

### Identifying invalid solutions

There's another file, `sudoku-invalids.csv` that contains invalid solutions of Sudokus. Your job is to read the solutions, and return only the ones that are invalid.

In [20]:
# Solution
from sudoku import is_valid, Board
def detect_invalid_solutions(filename):
    solutions = read_sudokus_from_csv(filename, read_solutions=True)
    return np.array([solution for solution in solutions if not is_valid(Board(solution))])

In [21]:
detect_invalid_solutions('sudoku-invalids.csv')

array([[[1, 7, 6, ..., 4, 5, 9],
        [5, 3, 8, ..., 6, 7, 2],
        [4, 9, 2, ..., 1, 3, 8],
        ...,
        [8, 1, 3, ..., 7, 4, 5],
        [7, 4, 9, ..., 2, 1, 6],
        [2, 6, 5, ..., 8, 9, 3]],

       [[9, 9, 5, ..., 6, 1, 3],
        [8, 4, 3, ..., 9, 5, 7],
        [7, 1, 6, ..., 8, 2, 4],
        ...,
        [6, 3, 4, ..., 2, 9, 1],
        [1, 8, 7, ..., 3, 6, 5],
        [2, 5, 9, ..., 7, 4, 8]],

       [[5, 8, 5, ..., 2, 1, 7],
        [3, 2, 1, ..., 9, 5, 6],
        [6, 9, 7, ..., 4, 8, 3],
        ...,
        [4, 5, 8, ..., 7, 3, 2],
        [9, 7, 3, ..., 6, 4, 1],
        [2, 1, 6, ..., 5, 9, 8]],

       ...,

       [[9, 1, 9, ..., 7, 2, 6],
        [3, 6, 5, ..., 8, 9, 4],
        [4, 7, 2, ..., 5, 1, 3],
        ...,
        [8, 3, 6, ..., 2, 4, 1],
        [5, 4, 9, ..., 6, 8, 7],
        [7, 2, 1, ..., 3, 5, 9]],

       [[5, 4, 2, ..., 7, 9, 8],
        [7, 9, 1, ..., 2, 6, 3],
        [8, 3, 6, ..., 5, 1, 4],
        ...,
        [2, 5, 3, ..., 

In [22]:
assert len(detect_invalid_solutions('sudoku-invalids.csv')) == 13

## Time to test!

Now it's time to move your code to `sudoku.py` and then run all the tests; if they're passing, you can move to the next step!

In [23]:
with open('test_part_4.py', 'w') as f:
    f.write('''
import numpy as np
from sudoku import (
    adapt_long_sudoku_line_to_array, read_sudokus_from_csv,
    detect_invalid_solutions)


def test_adapt_long_sudoku_lines():
    line = '067050010084309000003080040090000205000621790700093600300400000020007153500800076'
    expected = np.array([
        [0, 6, 7, 0, 5, 0, 0, 1, 0],
        [0, 8, 4, 3, 0, 9, 0, 0, 0],
        [0, 0, 3, 0, 8, 0, 0, 4, 0],
        [0, 9, 0, 0, 0, 0, 2, 0, 5],
        [0, 0, 0, 6, 2, 1, 7, 9, 0],
        [7, 0, 0, 0, 9, 3, 6, 0, 0],
        [3, 0, 0, 4, 0, 0, 0, 0, 0],
        [0, 2, 0, 0, 0, 7, 1, 5, 3],
        [5, 0, 0, 8, 0, 0, 0, 7, 6]])
    assert np.array_equal(adapt_long_sudoku_line_to_array(line), expected)


def test_read_csv_puzzles():
    expected = np.array([[
        [0, 8, 3, 2, 0, 0, 0, 9, 6],
        [2, 0, 0, 0, 3, 0, 7, 0, 4],
        [0, 0, 7, 9, 1, 5, 0, 0, 0],
        [4, 0, 2, 3, 9, 0, 0, 0, 8],
        [0, 1, 0, 0, 0, 4, 0, 6, 0],
        [0, 6, 9, 8, 7, 0, 0, 0, 0],
        [0, 0, 0, 4, 0, 0, 0, 0, 7],
        [5, 0, 0, 0, 6, 0, 2, 8, 0],
        [0, 7, 0, 0, 5, 0, 9, 0, 0]],

       [[8, 0, 3, 0, 0, 0, 2, 7, 0],
        [4, 0, 9, 0, 0, 8, 0, 0, 0],
        [7, 0, 0, 0, 2, 4, 0, 9, 6],
        [0, 0, 0, 0, 0, 6, 9, 1, 5],
        [0, 0, 1, 8, 0, 2, 0, 0, 0],
        [0, 3, 0, 7, 5, 0, 0, 0, 0],
        [0, 5, 4, 0, 0, 0, 0, 6, 0],
        [6, 0, 8, 1, 0, 0, 0, 0, 3],
        [3, 7, 2, 0, 0, 9, 1, 4, 0]]])
    assert np.array_equal(read_sudokus_from_csv('sudoku_test_valid.csv'), expected)


def test_read_csv_solutions():
    expected = np.array([[
        [1, 8, 3, 2, 4, 7, 5, 9, 6],
        [2, 9, 5, 6, 3, 8, 7, 1, 4],
        [6, 4, 7, 9, 1, 5, 8, 3, 2],
        [4, 5, 2, 3, 9, 6, 1, 7, 8],
        [7, 1, 8, 5, 2, 4, 3, 6, 9],
        [3, 6, 9, 8, 7, 1, 4, 2, 5],
        [9, 2, 1, 4, 8, 3, 6, 5, 7],
        [5, 3, 4, 7, 6, 9, 2, 8, 1],
        [8, 7, 6, 1, 5, 2, 9, 4, 3]],

       [[8, 6, 3, 9, 1, 5, 2, 7, 4],
        [4, 2, 9, 6, 7, 8, 3, 5, 1],
        [7, 1, 5, 3, 2, 4, 8, 9, 6],
        [2, 8, 7, 4, 3, 6, 9, 1, 5],
        [5, 4, 1, 8, 9, 2, 6, 3, 7],
        [9, 3, 6, 7, 5, 1, 4, 8, 2],
        [1, 5, 4, 2, 8, 3, 7, 6, 9],
        [6, 9, 8, 1, 4, 7, 5, 2, 3],
        [3, 7, 2, 5, 6, 9, 1, 4, 8]]])

    assert np.array_equal(read_sudokus_from_csv('sudoku_test_valid.csv', read_solutions=True), expected)


def test_detect_invalid_solutions():
    expected = np.array([[
        [8, 6, 3, 9, 1, 5, 5, 7, 4],
        [4, 2, 9, 6, 7, 8, 3, 5, 1],
        [7, 1, 5, 3, 2, 4, 8, 9, 6],
        [2, 8, 7, 4, 3, 6, 9, 1, 5],
        [5, 4, 1, 8, 9, 2, 6, 3, 7],
        [9, 3, 6, 7, 5, 1, 4, 8, 2],
        [1, 5, 4, 2, 8, 3, 7, 6, 9],
        [6, 9, 8, 1, 4, 7, 5, 2, 3],
        [3, 7, 2, 5, 6, 9, 1, 4, 8]]])
    assert np.array_equal(detect_invalid_solutions('sudoku_test_invalid.csv'), expected)
''')

In [24]:
!pytest test_part_4.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.5.0
rootdir: /content
plugins: typeguard-4.3.0, anyio-3.7.1
collected 4 items                                                                                  

test_part_4.py ....                                                                          [100%]

======================================== 4 passed in 0.29s =========================================
